# Demo

use the L1 GW170817 data to demonstrate (i) glitch removal, (ii) bandpass filtering, and (iii) whitening in the wavelet domain.

In [3]:
import jax.numpy as jnp
jnp.exp(jnp.array([k for k in range(10)]))  # Example, replace with actual logic

Array([1.0000000e+00, 2.7182817e+00, 7.3890562e+00, 2.0085537e+01,
       5.4598152e+01, 1.4841316e+02, 4.0342880e+02, 1.0966332e+03,
       2.9809580e+03, 8.1030840e+03], dtype=float32)